In [1]:
#Подключим нужные библиотеки
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import LocalOutlierFactor
from catboost import CatBoostRegressor, Pool

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

from scipy.stats import randint, uniform

Так как ранее данные уже были проанализированны и очишенны от выбросов и неэффективгых признаков сразу приступаем к построению моделей.

In [45]:
#Загрузи данные
CC50_df = pd.read_csv('./data/CC50.csv')
display(CC50_df)

,Unnamed: 0,"CC50, mM",MaxEStateIndex,MinAbsEStateIndex,qed,SPS,MinPartialCharge,MaxAbsPartialCharge,MinAbsPartialCharge,FpDensityMorgan1,...,fr_oxime,fr_piperzine,fr_priamide,fr_pyridine,fr_quatN,fr_sulfide,fr_sulfonamd,fr_thiazole,fr_thiophene,fr_urea
0,0,175.482382,5.094096,0.387225,0.417362,42.928571,-0.293526,0.293526,0.038844,0.642857,...,0,0,0,0,0,0,0,0,0,0
1,1,5.402819,3.961417,0.533868,0.462473,45.214286,-0.313407,0.313407,0.012887,0.607143,...,0,0,0,0,0,0,0,0,0,0
2,2,161.142320,2.627117,0.543231,0.260923,42.187500,-0.325573,0.325573,0.094802,0.562500,...,0,0,0,0,2,0,0,0,0,0
3,3,107.855654,5.097360,0.390603,0.377846,41.862069,-0.293526,0.293526,0.038844,0.620690,...,0,0,0,0,0,0,0,0,0,0
4,4,139.270991,5.150510,0.270476,0.429038,36.514286,-0.257239,0.257239,0.062897,0.600000,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
885,996,34.999650,12.934891,0.048029,0.382752,49.133333,-0.468587,0.468587,0.317890,1.133333,...,0,0,0,0,0,0,0,0,0,0
886,997,33.999415,13.635345,0.030329,0.369425,44.542857,-0.467493,0.467493,0.327562,1.085714,...,0,0,0,0,0,0,0,0,0,0
887,998,33.999458,13.991690,0.026535,0.284923,41.973684,-0.467485,0.467485,0.327887,1.157895,...,0,0,0,0,0,1,0,0,0,0
888,999,32.999644,13.830180,0.146522,0.381559,39.000000,-0.468755,0.468755,0.312509,0.756757,...,0,0,0,0,0,0,0,0,0,0


In [46]:
#Выведем целевой признак
target = CC50_df['CC50, mM']
features = CC50_df.drop(['CC50, mM', 'Unnamed: 0'], axis=1)

In [6]:
#Выполним стандартизацию данных
scaler = StandardScaler()
features = scaler.fit_transform(features)

In [7]:
#Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [9]:
#Опишем функцию печати метрик модели
def print_model_metrics(iTest, iPredict):
    # Средняя абсолютная ошибка (MAE)
    mae = mean_absolute_error(iTest, iPredict)
    print('{0:4} = {1}'.format('MAE', mae))
    
    # Среднеквадратичная ошибка (MSE)
    mse = mean_squared_error(iTest, iPredict)
    print('{0:4} = {1}'.format('MSE', mse))
    
    # Корень из среднеквадратичной ошибки (RMSE)
    rmse = np.sqrt(mse)
    print('{0:4} = {1}'.format('RMSE', rmse))
    
    # Коэффициент детерминации (R2)
    r2 = r2_score(iTest, iPredict)
    print('{0:4} = {1}'.format('R2', r2))

<h2>Модель линейной регрессии</h2>

In [10]:
model = LinearRegression()
model.fit(X_train, y_train)

# Предсказание
y_pred = model.predict(X_test)
print('Линейная регрессия:')
print_model_metrics(y_test, y_pred)

Линейная регрессия:
MAE  = 351.28504489765265
MSE  = 204901.02299768955
RMSE = 452.65994189644124
R2   = 0.47881812591769135


<h2>Модель DecisionTree</h2>

In [19]:
model = DecisionTreeRegressor(
    max_depth=5,
    min_samples_split=10,
    min_samples_leaf=3,
    random_state=42
)
model.fit(X_train, y_train)

# Предсказание
y_pred = model.predict(X_test)
print('Дерево решений:')
print_model_metrics(y_test, y_pred)

Дерево решений:
MAE  = 337.2654478908617
MSE  = 227427.28540724766
RMSE = 476.89336901161425
R2   = 0.42152080506041223


<h2>Модель Random Forest</h2>

In [31]:
#Обучение модели
model = RandomForestRegressor(
    n_estimators=40,
    max_depth=10,
    random_state=42
)
model.fit(X_train, y_train)

# Предсказание
y_pred = model.predict(X_test)
print('Ансамбль деревьев:')
print_model_metrics(y_test, y_pred)

Ансамбль деревьев:
MAE  = 299.97870607560026
MSE  = 175769.3898460378
RMSE = 419.2486014836994
R2   = 0.5529167269833745


<h2>Модель SVR</h2>

In [34]:
# Обучение модели
model = SVR(kernel='rbf', C=2000, epsilon=0.1)
model.fit(X_train, y_train)

# Предсказание
y_pred = model.predict(X_test)
print('Регрессия по опорным векторам:')
print_model_metrics(y_test, y_pred)

Регрессия по опорным векторам:
MAE  = 296.5829403107721
MSE  = 203537.90742850286
RMSE = 451.1517565393078
R2   = 0.4822853176210119


<h2>Модель KNN</h2>

In [41]:
# Обучение модели
model = KNeighborsRegressor(
    n_neighbors=10,
    weights='distance'
)
model.fit(X_train, y_train)

# Предсказание
y_pred = model.predict(X_test)
print('Регрессия по K-ближайшим соседям:')
print_model_metrics(y_test, y_pred)

Регрессия по K-ближайшим соседям:
MAE  = 303.01235590754965
MSE  = 207263.83964784694
RMSE = 455.2623854963717
R2   = 0.47280811585612226


In [42]:
model = CatBoostRegressor(
    iterations=100,
    learning_rate=0.4,
    depth=10,
    loss_function='RMSE',
    verbose=100,
    random_state=42
)

# Обучение модели
model.fit(X_train, y_train, eval_set=(X_test, y_test))

# Предсказание
y_pred = model.predict(X_test)
print('Регрессия CatBoost:')
print_model_metrics(y_test, y_pred)

0:	learn: 541.7983865	test: 552.0138155	best: 552.0138155 (0)	total: 65.7ms	remaining: 6.51s
99:	learn: 192.5659704	test: 421.0355155	best: 419.2156279 (72)	total: 1.73s	remaining: 0us

bestTest = 419.2156279
bestIteration = 72

Shrink model to first 73 iterations.
Регрессия CatBoost:
MAE  = 281.86351204201975
MSE  = 175741.7425253121
RMSE = 419.2156277207615
R2   = 0.5529870501189957


<b>Для лучшей модели попробуем автоматический подбор гипер параметров</b>

In [44]:
param_dist = {
    'n_estimators': randint(2, 100),
    'max_depth': randint(2, 20),
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 10),
    'max_features': ['sqrt', 'log2', None],
    'criterion': ['friedman_mse', 'absolute_error', 'poisson'],
    'max_samples': uniform(0.1, 0.9),
}
random_search = RandomizedSearchCV(
    estimator= RandomForestRegressor(random_state=42),
    param_distributions=param_dist,
    n_iter=500,
    scoring='r2',
    cv=5,
    random_state=42,
    n_jobs=-10
)

random_search.fit(X_train, y_train)

best_params = random_search.best_params_
print("Лучшие параметры:", best_params)

best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)

print('Лучшие метрики Random Forest при подборе:')
print_model_metrics(y_test, y_pred)


Лучшие параметры: {'criterion': 'friedman_mse', 'max_depth': 19, 'max_features': None, 'max_samples': 0.6930315477191362, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 68}
Лучшие метрики Random Forest при подборе:
MAE  = 296.93447936484074
MSE  = 177702.54159375318
RMSE = 421.5477927753307
R2   = 0.5479996034081978


<h2>Выводы</h2>

Для прогнозирования показателя <b>СC50</b>, были использованы основные модели классического ML, такие как:

* Linear regression;
* DecisionTree;
* Random Forest;
* SVR;
* KNN.

А так же готовое решение CatBoostRegressor.

Наилучшую метрику R2 показала модель Random Forest со значением 0.55 и CatBoostRegressor с аналогичным результатом, что считается достаточно низким. Возможно не хватает данных для улучшения прогнозирования.

Для подбора наилучших гипер параметров был использован алгоритм случайного поиска. Но ручной подбор гипер параметров оказался лучше.